In [377]:
import numpy as np

In [378]:
class LeakyRelu:
    def __init__(self, alpha=0.1):
        self.alpha = alpha
    
    def output(self, x):
        return np.maximum(x*self.alpha, x)
    
    def derivative(self, x):
        return np.where(x>0, 1, self.alpha) 

In [379]:
class Neuron:
    def __init__(self, input_size):
        self.input_vec = np.empty(input_size)
        self.output = np.empty(1)
        # dtype is np.float64 by default
        self.weights = np.random.uniform(-0.3, 0.3, input_size)
        self.local_grad = np.empty(1)
    
    def update_weights(self, etta):
        delta_weights = np.multiply(etta, self.local_grad)
        self.weights = self.weights + delta_weights
        
    def call(self, input_vector):
        self.input_vec = input_vector
        self.output = np.dot(input_vector, self.weights)
        return self.output
    
    @property
    def w(self):
        return self.weights[1:]
    
    @property  
    def b(self):
        return self.weights[0]

In [380]:
class NeuralLayer:
    def __init__(self, input_dimension: int, output_dimension: int, activation_function, bias=True):
        # bias is True by default and in this code I did not change it for hidden layers
        self.input_dimension = input_dimension+1
        self.neuron_count = output_dimension+1 if bias else output_dimension
        self.neurons = [Neuron(self.input_dimension) for _ in range(self.neuron_count)]
        self.a_f = activation_function
        self.localgrads_in_w = np.empty((self.neuron_count, self.input_dimension))
    
    def calculate_outputs(self, x_input):
        outputs = []      
        for neuron in self.neurons:
            output = self.a_f.output(neuron.call(x_input))
            outputs.append(output)
        
        return np.array(outputs)
    
    def calculate_outputlayer_localgrad(self, loss):
        for i, neuron in enumerate(self.neurons):
            neuron.local_grad = np.multiply(loss, self.a_f.derivative(neuron.output))
            self.localgrads_in_w[i] = (np.multiply(neuron.local_grad, neuron.weights))
        
    def calculate_hiddenlayer_localgrad(self, next_layer):
        sum_nexlayer_localgrads = np.sum(next_layer.localgrads_in_w, axis=0)
        for i, neuron in enumerate(self.neurons):
            neuron.local_grad = np.multiply(self.a_f.derivative(neuron.output), sum_nexlayer_localgrads[i])
    
    def update_weights(self, etta):
        for neuron in self.neurons:
            neuron.update_weights(etta)
    
    @property
    def weights(self):
        return [neuron.weights for neuron in self.neurons]
    
    @property
    def neuron_list(self):
        return list(self.neurons)

In [398]:
class NeuralNetwork:
    def __init__(self):
        self.neural_layers = []
    
    def sequential(self, neural_layers: list):
        self.neural_layers = neural_layers
    
    def zero_grad(self):
        for layer in self.neural_layers:
            layer.zero_grad()
        
    def forward(self, X):
        input_vector = np.concatenate((np.array(X), np.array([1])), axis=0)
        for layer in self.neural_layers:
            output_vector = layer.calculate_outputs(input_vector)
            input_vector = output_vector
        return output_vector
    
    def backward(self, loss):
        self.neural_layers[-1].calculate_outputlayer_localgrad(loss)
        
        reverse_order = self.neural_layers[::-1]
        for i, layer in enumerate(reverse_order[1:], 1): 
            next_layer = reverse_order[i-1]
            layer.calculate_hiddenlayer_localgrad(next_layer)
    
    def update_weights(self, etta):
        for layer in self.neural_layers:
            layer.update_weights(etta)
    
    
    def loss(self, desired_output, predicted_output):
        return np.mean(np.subtract(np.array(desired_output), np.array(predicted_output)))
    
    @property
    def layers(self):
        return self.neural_layers
    

In [413]:
model = NeuralNetwork()
model.sequential([
    NeuralLayer(2,3, LeakyRelu()),
    NeuralLayer(3,2, LeakyRelu(), bias=False),
])


In [414]:
# hyper_parameters
num_epochs = 20
learning_rate = 0.1

In [415]:
for _ in range(num_epochs):
    y_pred = model.forward(np.array([1,2]))
    loss = model.loss([1,1], y_pred)
    print(loss)
    model.backward(loss)
    model.update_weights(learning_rate)

0.9898213275080514
0.9823955429097722
0.9688867222457107
0.9436886896237235
0.9022311414157838
0.838656750997308
0.6740262398360437
0.4996334864689257
0.3430840179044615
0.2209158826518658
0.1355373826577284
0.08044819230196587
0.04675208479344639
0.026823112207944977
0.01527320652937092
0.00865864117583448
0.004896450670173524
0.0027649967621619087
0.0015601186689720992
0.0008798784746727462
